In [2]:
import torch
import multiprocessing

# Set the start method to 'spawn' for multiprocessing
if __name__ == "__main__":
    multiprocessing.set_start_method('spawn', force=True)


In [3]:
pip install transformers==4.52.0 datasets==3.6.0 evaluate jiwer torchaudio  matplotlib seaborn soundfile libsndfile1

ERROR: Could not find a version that satisfies the requirement libsndfile1 (from versions: none)

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip
ERROR: No matching distribution found for libsndfile1
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install accelerate


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install torchcodec


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
!pip install --upgrade torch torchvision torchaudio


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip


In [7]:
!apt-get update -y
!apt-get install -y ffmpeg


Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 http://security.ubuntu.com/ubuntu jammy-security InRelease               
Hit:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease                 
Hit:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease               
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease   
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 100 not upgraded.


In [8]:
import os
from datasets import Dataset
from torch.nn.utils.rnn import pad_sequence

import evaluate
import numpy as np
import jiwer

from transformers import TrainingArguments, Trainer
import torch
import matplotlib.pyplot as plt

In [9]:
# from datasets import load_dataset

# # Try loading the dataset without streaming
# dataset = load_dataset("facebook/multilingual_librispeech", "french")

# dataset

In [10]:
# dataset = dataset["train"].remove_columns(['original_path', 'begin_time', 'end_time', 'audio_duration', 'chapter_id', 'file', 'id'])
# dataset

In [11]:
from datasets import load_dataset
from huggingface_hub import notebook_login
notebook_login()

cv_17 = load_dataset("mozilla-foundation/common_voice_17_0", "fr", split="train")

In [12]:
dataset = cv_17.remove_columns(['path','client_id','up_votes','down_votes','age','gender','accent','locale','segment','variant'])
dataset

Dataset({
    features: ['audio', 'sentence'],
    num_rows: 558054
})

In [13]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration, WhisperTokenizer, WhisperFeatureExtractor
model_id = 'openai/whisper-small'

feature_extractor = WhisperFeatureExtractor.from_pretrained(model_id)
 
tokenizer = WhisperTokenizer.from_pretrained(model_id, language='French', task='transcribe')
 
processor = WhisperProcessor.from_pretrained(model_id, language='French', task='transcribe')


In [14]:
filtered_data = dataset.select(range(1))
def print_first_audio(batch): 
    first_audio = batch["audio"][0] 
    print(f"Sampling rate: {first_audio['sampling_rate']}")
    print(f"array:{first_audio['array']}")
    return batch
filtered_data.map(print_first_audio, batched=True, batch_size=1)


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Sampling rate: 48000
array:[ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ... -2.74169997e-05
 -8.92494791e-06  2.03383570e-06]


Dataset({
    features: ['audio', 'sentence'],
    num_rows: 1
})

In [15]:
from datasets import Audio

dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))

In [16]:
filtered_data = dataset.select(range(1))
def print_first_audio(batch): 
    first_audio = batch["audio"][0] 
    print(f"Sampling rate: {first_audio['sampling_rate']}")
    print(f"array:{first_audio['array']}")
    return batch
filtered_data.map(print_first_audio, batched=True, batch_size=1)


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Sampling rate: 16000
array:[ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ... -3.92589482e-06
 -6.20179480e-06 -1.21230814e-05]


Dataset({
    features: ['audio', 'sentence'],
    num_rows: 1
})

In [17]:
# filtered_data = dataset.select(range(1000))

def prepare_data(batch):
    # Extract audio arrays
    audio_arrays = [a["array"] for a in batch["audio"]]

    # Process multiple audio clips at once
    batch["input_features"] = feature_extractor(audio_arrays,sampling_rate=16000).input_features

    # Tokenize transcripts
    batch["labels"] = tokenizer(batch["sentence"]).input_ids

    return batch


# Map with batching + multiprocessing
tokenized_dataset = dataset.map(
    prepare_data,
    remove_columns=["audio", "sentence"],
    batched=True,
    batch_size=64
)


Map:   0%|          | 0/558054 [00:00<?, ? examples/s]

In [18]:
from datasets import Dataset

tokenized_dataset.save_to_disk("./tokenized_dataset")

Saving the dataset (0/1073 shards):   0%|          | 0/558054 [00:00<?, ? examples/s]

In [19]:
# from datasets import load_from_disk

# # Replace with your dataset path
# tokenized_dataset = load_from_disk("./tokenized_dataset")
# tokenized_dataset

In [ ]:
# from datasets import ClassLabel

# # Get the unique speaker_ids as strings
# unique_speaker_ids = list(map(str, sorted(set(tokenized_dataset['speaker_id']))))

# # Create a ClassLabel object using these unique string labels
# filtered_dataset = tokenized_dataset.cast_column('speaker_id', ClassLabel(names=unique_speaker_ids))
# filtered_dataset

In [ ]:
# filtered_dataset['speaker_id']

In [ ]:
# from collections import defaultdict

# # Step 1: Count speaker occurrences in one pass using map
# def count_speakers(batch):
#     for speaker in batch["speaker_id"]:
#         speaker_freq[speaker] += 1
#     return batch

# # Use a defaultdict to store counts
# speaker_freq = defaultdict(int)

# # Run over the full dataset once (very fast)
# _ = filtered_dataset.map(count_speakers, batched=True, batch_size=1000)

# # Step 2: Filter dataset to keep only speaker_ids with count >= 2
# valid_speakers = set(speaker for speaker, count in speaker_freq.items() if count >= 10)



In [ ]:
# valid_speakers

In [ ]:
# def batch_filter(batch):
#     return [speaker in valid_speakers for speaker in batch['speaker_id']]

# filtered_dataset = filtered_dataset.filter(batch_filter, batched=True, batch_size=10000)

In [ ]:
# filtered_dataset

In [20]:
train_val_split  = tokenized_dataset.train_test_split(test_size=0.1)#, stratify_by_column='speaker_id')
train_dataset = (train_val_split ['train'])#.remove_columns(['speaker_id'])
eval_dataset = (train_val_split ['test'])#.remove_columns(['speaker_id'])


In [21]:
import evaluate
wer_metric = evaluate.load("wer")

def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)
    # for i in range(min(5, len(pred_str))):  # Limit to first 5 examples to avoid too much output
    #     print(f"\nExample {i+1}:")
    #     print(f"Prediction: {pred_str[i]}")
    #     print(f"Reference : {label_str[i]}")
    wer = 100 * wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}


In [22]:
# import torch
# from dataclasses import dataclass
# from typing import Any, Dict, List, Union

# @dataclass
# class DataCollatorSpeechSeq2SeqWithPadding:
#     processor: Any
#     decoder_start_token_id: int

#     def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
#         # split inputs and labels since they have to be of different lengths and need different padding methods
#         # first treat the audio inputs by simply returning torch tensors
#         input_features = [{"input_features": feature["input_features"]} for feature in features]
#         batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

#         # get the tokenized label sequences
#         label_features = [{"input_ids": feature["labels"]} for feature in features]
#         # pad the labels to max length
#         labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

#         # replace padding with -100 to ignore loss correctly
#         labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

#         # if bos token is appended in previous tokenization step,
#         # cut bos token here as it's append later anyways
#         if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
#             labels = labels[:, 1:]

#         batch["labels"] = labels

#         return batch


# data_collator = DataCollatorSpeechSeq2SeqWithPadding(
#     processor=processor,
#     decoder_start_token_id=model.config.decoder_start_token_id,
# )

In [23]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

model.generation_config.language = "french"
model.generation_config.task = "transcribe"

model.generation_config.forced_decoder_ids = None

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json: 0.00B [00:00, ?B/s]

In [ ]:
from data_collator import DataCollatorSpeechSeq2SeqWithPadding
import os
data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=model.config.decoder_start_token_id,
)

from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

os.environ["WANDB_DISABLED"] = "true"

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-small-hi",  # change to a repo name of your choice
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    num_train_epochs=3,
    warmup_steps=500,
    dataloader_num_workers=8,
    gradient_checkpointing=False,
    fp16=True,
    eval_strategy="epoch",
    no_cuda=False, 
    per_device_eval_batch_size=16,
    predict_with_generate=True,
    generation_max_length=225,
    save_strategy='epoch',
    # eval_steps=10,
    logging_steps=10,
    load_best_model_at_end=True,
    save_total_limit=1,
    metric_for_best_model="wer",
    greater_is_better=False,
)



trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

trainer.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/tmp/ipykernel_3603/4067385339.py:38: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss,Wer
1,0.225100,0.223927,13.350089
2,0.159600,0.205603,12.183619


You have passed task=transcribe, but also have set `forced_decoder_ids` to [[1, 50259], [2, 50359], [3, 50363]] which creates a conflict. `forced_decoder_ids` will be ignored in favor of task=transcribe.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3464: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 2

In [ ]:
trainer.save_model("./asr_model6")
processor.save_pretrained("./asr_model6")
